# Fully Connected Neural Network

Notebook inspired by https://github.com/aymericdamien/TensorFlow-Examples/

Example is using the [MNIST database of handwritten digits](http://yann.lecun.com/exdb/mnist/)

In [ ]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden = 256
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

tf.reset_default_graph()

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

## Exercise 1

Define a function that builds a fully connected neural network. You will need to complete these steps:

1. define 4 `tf.Variable` with the appropriate shapes for W, b, W_out, b_out. Initialize them with random values.

- define a super simple network with 1 layer that performs the operation:

        relu(x * W + b)

- define an output layer that performs the operation:

        softmax(x * W_out + b_out)

- encapsulate these in a function called `dnn` that takes `x` as input and returns the output layer

In [ ]:
# Create model
def dnn(x, n_hidden_1):
    # hidden layer
    W = tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W')
    b = tf.Variable(tf.random_normal([n_hidden_1]), name='b')
    layer_1 = tf.add(tf.matmul(x, W), b)
    layer_1 = tf.nn.relu(layer_1)
    
    # Output layer with linear activation
    W_out = tf.Variable(tf.random_normal([n_hidden_1, n_classes]), name='W_out')
    b_out = tf.Variable(tf.random_normal([n_classes]), name='b_out')
    out_layer = tf.matmul(layer_1, W_out) + b_out
    return out_layer

In [ ]:
# Construct model
pred = dnn(x, n_hidden)

What does the graph look like for this network?

In [ ]:
g = tf.get_default_graph()
[op.name for op in g.get_operations()]

In [ ]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

## Exercise 2

1. modify the `dnn` function adding a second hidden layer also with `relu` activation

In [ ]:
# Create model
def dnn_2(x, n_hidden_1, n_hidden_2):
    # hidden layer 1
    W = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
    b = tf.Variable(tf.random_normal([n_hidden_1]))
    layer_1 = tf.add(tf.matmul(x, W), b)
    layer_1 = tf.nn.relu(layer_1)
    
    # hidden layer 2
    W2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
    b2 = tf.Variable(tf.random_normal([n_hidden_2]))
    layer_2 = tf.add(tf.matmul(layer_1, W2), b2)
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with linear activation
    W_out = tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    b_out = tf.Variable(tf.random_normal([n_classes]))
    out_layer = tf.matmul(layer_2, W_out) + b_out
    return out_layer

In [ ]:
pred = dnn_2(x, 512, 256)
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})